In [1]:
import sys
import torch
import wandb
sys.path.append('..')
import pytorch_lightning as pl
from utils.data_modules.super_resolution import EEGSuperResolutionDataModule
from models.trainers.super_resolution import SuperResolutionTrainerModel, SuperResolutionTrainerCallback

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


S:\PolySecLabProjects\eeg-image-decoding\data\all-joined-1\eeg\epochs


In [2]:
dm = EEGSuperResolutionDataModule(
    input_channels=['Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2'],
    output_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=32, 
    num_workers=4
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 1
subject = 1
session = 1
input_channels = sample_data['input']['channel_names']
output_channels = sample_data['output']['channel_names']
timesteps = sample_data['input']['num_timesteps']

Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320
Original dataframe size: 640


In [ ]:
lightning_model = SuperResolutionTrainerModel(input_channels, output_channels, timesteps)
    
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="S:\\PolySecLabProjects\\eeg-image-decoding\\updated-code\\models\\check_points\\super_resolution",
    filename=f"subj{subject}_session{session}_{{epoch}}",
    every_n_epochs=5,
    save_top_k=-1
)

results_callback = SuperResolutionTrainerCallback()

logger = pl.loggers.WandbLogger(project="eeg_super_resolution")

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[checkpoint_callback, results_callback],
    logger=logger,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.fit(lightning_model, dm)
# trainer.test(lightning_model, dm)

if logger and isinstance(logger, pl.loggers.WandbLogger):
    logger.finalize('success')
    wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: dubs2310 (dubs2310-cal-poly-pomona) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                           | Params | Mode 
----------------------------------------------------------------------------------
0 | super_resolution_model | EEGSuperResolutionDefiner      | 492 K  | train
1 | mean_train_loss        | MeanMetric                     | 0      | train
2 | mean_val_loss          | MeanMetric                     | 0      | train
3 | mean_test_loss         | MeanMetric                     | 0      | train
4 | mean_test_snr          | SignalNoiseRatio               | 0      | train
5 | mean_test_mae          | MeanAbsoluteError              | 0      | train
6 | mean_test_mse          | MeanSquaredError               | 0      | train
7 | mean_test_nrmse        | NormalizedRootMeanSquaredError | 0      | train
8 | mean_test_pearson      | PearsonCorrCoef                | 0      | train
----------------------------------------------------------------------------------
492 K     Trainable p

Original dataframe size: 640
Epoch 0: 100%|██████████| 90/90 [06:43<00:00,  0.22it/s, v_num=nfa1]       

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 90/90 [06:43<00:00,  0.22it/s, v_num=nfa1]


TypeError: isinstance expected 2 arguments, got 1

In [5]:
if logger and isinstance(logger, pl.loggers.WandbLogger):
    logger.finalize('success')
    wandb.finish()

epoch,▁▁▁
train_batch_loss,▁
train_loss,▁
trainer/global_step,▁██
val_loss,▁
epoch,0
train_batch_loss,120.7109
train_loss,130.10382
trainer/global_step,89
val_loss,127.13895


In [6]:
results = []
for epoch in range(len(results_callback.train_losses)):
    epoch_results = {
        "epoch": epoch + 1,
        "train_loss": results_callback.train_losses[epoch],
        "val_loss": results_callback.val_losses[epoch] if epoch < len(results_callback.val_losses) else 0
    }
    results.append(epoch_results)

if results_callback.test_metrics:
    results.append({"test_metrics": results_callback.test_metrics})

results

[{'epoch': 1, 'train_loss': 0, 'val_loss': 0}]